In [1]:
import requests
import pandas as pd
import glob
import time
import os
from dotenv import load_dotenv
import re

In [ ]:
import nltk
nltk.download('popular')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\kwind\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\kwind\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\kwind\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\kwind\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\kwind\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
#récupération api clé
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [3]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = glob.glob("CSV_Categories/*.csv")

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df = df.sort_values(by=['video_id', 'views'])
df.drop_duplicates(subset=['video_id'], keep="last", inplace=True)
df.reset_index(inplace=True)
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 13940 entries, 0 to 13939
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   index         13940 non-null  int64
 1   video_id      13940 non-null  str  
 2   title         13940 non-null  str  
 3   description   9279 non-null   str  
 4   channel       13940 non-null  str  
 5   published_at  13940 non-null  str  
 6   duration      13924 non-null  str  
 7   views         13940 non-null  int64
 8   likes         13940 non-null  int64
 9   comments      13940 non-null  int64
 10  channel_id    13940 non-null  str  
 11  category_id   13940 non-null  int64
 12  language      13794 non-null  str  
dtypes: int64(5), str(8)
memory usage: 1.4 MB


In [4]:
df_top_chaines = df.groupby('channel_id')['views'].sum().reset_index()
df_top_chaines = df_top_chaines[df_top_chaines['views'] > 10000]
df_top_chaines.info()

<class 'pandas.DataFrame'>
Index: 4113 entries, 0 to 8038
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   channel_id  4113 non-null   str  
 1   views       4113 non-null   int64
dtypes: int64(1), str(1)
memory usage: 96.4 KB


In [5]:
# Récupération des infos des chaines
all_channel_data = []

# On boucle sur le DataFrame par paquets de 50 IDs
for i in range(0, len(df_top_chaines), 50):
    batch_ids = df_top_chaines['channel_id'].iloc[i:i+50].tolist()
    ids_string = ",".join(batch_ids) 
    
    params = {
        "part": "snippet,contentDetails,statistics,topicDetails,status",
        "id": ids_string,
        "key": API_KEY
    }

    r = requests.get(f"https://www.googleapis.com/youtube/v3/channels", params=params)
    data = r.json()

    if "items" in data:
        for item in data["items"]:
            snippet = item.get("snippet", {})
            statistics = item.get("statistics", {})
            contentDetails = item.get("contentDetails", {})
            topicDetails = item.get("topicDetails", {})
            status = item.get("status", {})

            channel_info = {
                "id": item.get("id"),
                "title": snippet.get("title"),
                "description": snippet.get("description"),
                "country": snippet.get("country"),
                "views": int(statistics.get("viewCount", 0)),
                "subscribers": int(statistics.get("subscriberCount", 0)),
                "nb_videos": int(statistics.get("videoCount", 0)),
                "uploads_playlist": contentDetails.get("relatedPlaylists", {}).get("uploads", ""),
                "topics": topicDetails.get("topicCategories", []) # Plus lisible que topicIds
            }
            all_channel_data.append(channel_info)

# Création du DataFrame final
df_channels = pd.DataFrame(all_channel_data)
df_channels.info()


<class 'pandas.DataFrame'>
RangeIndex: 4110 entries, 0 to 4109
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                4110 non-null   str   
 1   title             4110 non-null   str   
 2   description       4110 non-null   str   
 3   country           3129 non-null   str   
 4   views             4110 non-null   int64 
 5   subscribers       4110 non-null   int64 
 6   nb_videos         4110 non-null   int64 
 7   uploads_playlist  4110 non-null   str   
 8   topics            4110 non-null   object
dtypes: int64(3), object(1), str(5)
memory usage: 289.1+ KB


In [6]:
# ne garder que les chaines fr, us, gb, ca, de
df_channels =  df_channels[(df_channels['country']=="FR")|(df_channels['country']=="US")|(df_channels['country']=="GB")|(df_channels['country']=="CA")].copy()
df_channels.info()

<class 'pandas.DataFrame'>
Index: 2035 entries, 3 to 4108
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                2035 non-null   str   
 1   title             2035 non-null   str   
 2   description       2035 non-null   str   
 3   country           2035 non-null   str   
 4   views             2035 non-null   int64 
 5   subscribers       2035 non-null   int64 
 6   nb_videos         2035 non-null   int64 
 7   uploads_playlist  2035 non-null   str   
 8   topics            2035 non-null   object
dtypes: int64(3), object(1), str(5)
memory usage: 159.0+ KB


In [7]:
# COLONNE HASHTAGS

# extraction des hashtags des colonnes title et description
text_title_description = df_channels['title'].fillna('') + " " + df_channels['description'].fillna('')
df_channels['hashtags'] = text_title_description.apply(lambda x: list(set(re.findall(r"#\w+", x))) or None)

# exploser la colonne hashtag pour compter les occurences
df_exploded = df_channels.explode('hashtags').copy()
df_exploded = df_exploded.dropna(subset=['hashtags'])

df_exploded['hashtags'].value_counts()


hashtags
#shorts            6
#1                 4
#humour            4
#roblox            3
#organisation      3
                  ..
#60hits            1
#livestream        1
#classicrock       1
#heavyrockmusic    1
#heavyrock         1
Name: count, Length: 495, dtype: int64

In [8]:
# COLONNE TOPICS

def extract_topic_name(topic_list):
    # Si la valeur est None ou n'est pas une liste/iterable
    if not isinstance(topic_list, list):
        return ""
    
    # Nettoyage de chaque URL dans la liste
    clean_list = [url.split('/')[-1].replace('_', ' ') for url in topic_list]
    return ", ".join(clean_list)

df_channels['topics'] = df_channels['topics'].apply(extract_topic_name)

In [55]:
# COLONNE KEYWORDS (uniquement pour chaines FR)

def clean_stem(str):
  items_to_remove= set(stopwords.words("french"))

  #tokenisation
  texte_tokens =  nltk.word_tokenize(str)

  tokens_clean = []

  # suppression stopwords
  for word in texte_tokens :
    word =  word.lower()
    #version bis : on ne supprime pas la ponctuation
    if word not in items_to_remove:
      tokens_clean.append(word)


  # stemming
  stem_en = SnowballStemmer("french")
  texte_clean_stem  = [stem_en.stem(word) for word in tokens_clean]
  return(' '.join(texte_clean_stem))

In [57]:
df_channels_fr =  df_channels[df_channels['country']=="FR"]
df_channels_fr.reset_index()
df_channels_fr['keywords'] = df_channels['description'].apply(clean_stem)
df_channels_fr.head()

,index,id,title,description,country,views,subscribers,nb_videos,uploads_playlist,topics,hashtags,keywords
2,7,UC-1FhqxA7zOPVG4fl2qBs_A,Roxas V6,Yo les amis c'est Roxas ^^ \nPassionné de jeux...,FR,3599518,19700,489,UU-1FhqxA7zOPVG4fl2qBs_A,,None,"yo amis c'est rox ^^ passion jeux vidéos , j'a..."
3,8,UC-6XX8GoB0LgINUjnuXWZrQ,ZigTik,Bienvenue sur notre chaîne de défis et de dive...,FR,7782549,70900,69,UU-6XX8GoB0LgINUjnuXWZrQ,,None,bienvenu chaîn def divert captiv ! 🎉 plong uni...
5,10,UC-4xdm_HWFZqaoZVbJ72o0w,Redouane Bougheraba TV,Allez Paf... tu trouveras ici sur ma chaîne Yo...,FR,203685091,558000,440,UU-4xdm_HWFZqaoZVbJ72o0w,,None,allez paf ... trouv ici chaîn youtub meilleur ...
6,11,UC-3UNIRUAEnjQSJVTz2Q2WA,MangaMotionStudio,🎬 MangaMotionStudio l’univers des AMV émotion...,FR,335491,3320,45,UU-3UNIRUAEnjQSJVTz2Q2WA,,"[#AnimeEdit, #AnimeVibes, #AMV, #EpicAMV, #Emo...",🎬 mangamotionstudio ’ univer amv émotionnel ép...
7,12,UC-KkVdOP86k1SDAWzFfw8nA,DEMONEXION,WESH LES FRÉROTS c'est .. TONTON DEMONEXION ! ...,FR,288202334,718000,492,UU-KkVdOP86k1SDAWzFfw8nA,,None,wesh frérot c'est .. tonton demonexion ! bon h...


In [9]:
df_channels.head()

,id,title,description,country,views,subscribers,nb_videos,uploads_playlist,topics,hashtags
3,UC-KkVdOP86k1SDAWzFfw8nA,DEMONEXION,WESH LES FRÉROTS c'est .. TONTON DEMONEXION ! ...,FR,288202334,718000,492,UU-KkVdOP86k1SDAWzFfw8nA,"Entertainment, Lifestyle (sociology)",None
4,UC-GI5LST5T3Gw93yZxjdFaw,DORETDEPLATINE,DOUBLE ALBUM TP SUR TP DISPONIBLE 👽💣\n✊ et mer...,FR,5790610567,7610000,377,UU-GI5LST5T3Gw93yZxjdFaw,"Hip hop music, Music, Pop music",None
6,UC-QCL3mRR48JKSFQ3uAUSgQ,TopSongs Hub,TopSongs Hub is your go-to music channel for t...,US,1364788,4090,43,UU-QCL3mRR48JKSFQ3uAUSgQ,"Electronic music, Pop music, Music, Christian ...",None
8,UC-3UNIRUAEnjQSJVTz2Q2WA,MangaMotionStudio,🎬 MangaMotionStudio l’univers des AMV émotion...,FR,335491,3320,45,UU-3UNIRUAEnjQSJVTz2Q2WA,"Entertainment, Music, Film, Pop music, Electro...","[#AnimeEdit, #CinematicAMV, #EmotionalAMV, #Ep..."
10,UC-4M8AN08hw39nn2v91VuMQ,CYRILmp4,J’explore la création !\n,FR,1347868034,5200000,1084,UU-4M8AN08hw39nn2v91VuMQ,"Lifestyle (sociology), Video game culture",None


In [10]:
df_channels.to_csv("chaines.csv",index=False,sep=',', encoding='utf-8-sig',quoting=1)